

```
# This is formatted as code
```

## Exploración, explicación y limpieza de datos

El dataset `Articles.csv` contiene un total de **2,692 instancias** y **4 columnas**: `Article`, `Date`, `Heading` y `NewsType`. Cada fila corresponde a una noticia publicada, su fecha, su titular y la categoría a la que pertenece (business o sports).

Durante la exploración verificamos que:
- **No existen valores nulos** en ninguna columna.
- El dataset está relativamente balanceado:
  - `sports`: 1,408 instancias
  - `business`: 1,284 instancias
- La columna que usamos para el modelo generativo es **Heading**, que contiene el titular de cada noticia.

También analizamos la **longitud de los titulares**, obteniendo:
- Longitud promedio: **8.15 palabras**
- Mínimo: 3 palabras
- Máximo: 19 palabras

Esto confirma que los titulares son secuencias cortas, ideales para entrenar un modelo generativo tipo LSTM.  
Ejemplos de titulares:

- "sindh govt decides to cut public transport fares..."
- "asia stocks up in new year..."
- "hong kong stocks open 0.66 percent lower..."

### Limpieza aplicada
Solo conservamos las columnas `Heading` y `NewsType`, eliminando columnas irrelevantes para el modelo generativo.  
También tokenizamos, creamos vocabulario, agregamos tokens especiales `<bos>`, `<eos>`, `<pad>` y `<unk>` y convertimos los titulares a secuencias numéricas de longitud fija.

Este preprocesamiento prepara los datos para entrenar un **modelo de lenguaje** con LSTM.


In [ ]:
# Importación de librerías y configuración
import pandas as pd
import numpy as np
import re
import random
import math
from collections import Counter

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Fijamos semilla para reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

Usando dispositivo: cpu


In [ ]:
# Carga del dataset
df = pd.read_csv("Articles.csv", encoding="latin-1")

# Vemos las primeras filas
df.head()

,Article,Date,Heading,NewsType
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business


In [ ]:
# Exploración del dataset

# Dimensiones del dataset
print("Shape del dataset:", df.shape)

# Nombre de columnas
print("\nColumnas:", df.columns.tolist())

# Info general (tipos de datos)
print("\nInformación del dataframe:")
print(df.info())

# Valores nulos por columna
print("\nValores nulos por columna:")
print(df.isna().sum())

# Distribución de clases de NewsType
print("\nDistribución de tipos de noticia (NewsType):")
print(df["NewsType"].value_counts())

Shape del dataset: (2692, 4)

Columnas: ['Article', 'Date', 'Heading', 'NewsType']

Información del dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2692 entries, 0 to 2691
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Article   2692 non-null   object
 1   Date      2692 non-null   object
 2   Heading   2692 non-null   object
 3   NewsType  2692 non-null   object
dtypes: object(4)
memory usage: 84.3+ KB
None

Valores nulos por columna:
Article     0
Date        0
Heading     0
NewsType    0
dtype: int64

Distribución de tipos de noticia (NewsType):
NewsType
sports      1408
business    1284
Name: count, dtype: int64


In [ ]:
# Análisis de longitud de los titulares
def simple_tokenize(text):
    """
    Tokenizador muy sencillo:
    - Pasa a minúsculas
    - Separa por palabras y signos de puntuación
    """
    return re.findall(r"\w+|\S", str(text).lower())

lengths = df["Heading"].apply(lambda x: len(simple_tokenize(x)))
print("Descripción estadística de la longitud de los titulares:")
print(lengths.describe())

print("\nEjemplos de titulares:")
for i in range(5):
    print(f"- {df['Heading'].iloc[i]}")

Descripción estadística de la longitud de los titulares:
count    2692.000000
mean        8.158990
std         1.798669
min         3.000000
25%         7.000000
50%         8.000000
75%         9.000000
max        19.000000
Name: Heading, dtype: float64

Ejemplos de titulares:
- sindh govt decides to cut public transport fares by 7pc kti rej
- asia stocks up in new year trad
- hong kong stocks open 0.66 percent lower
- asian stocks sink euro near nine year 
- us oil prices slip below 50 a barr


In [ ]:
# Limpieza básica de datos

# En este dataset no hay nulos en Heading, pero en general:
df = df.dropna(subset=["Heading"]).reset_index(drop=True)

# Nos quedamos solo con columnas que vamos a usar
df = df[["Heading", "NewsType"]]

print("Shape después de limpieza:", df.shape)
df.head()

Shape después de limpieza: (2692, 2)


,Heading,NewsType
0,sindh govt decides to cut public transport far...,business
1,asia stocks up in new year trad,business
2,hong kong stocks open 0.66 percent lower,business
3,asian stocks sink euro near nine year,business
4,us oil prices slip below 50 a barr,business


In [ ]:
# Construcción de vocabulario

BOS_TOKEN = "<bos>"
EOS_TOKEN = "<eos>"
PAD_TOKEN = "<pad>"
UNK_TOKEN = "<unk>"

special_tokens = [PAD_TOKEN, BOS_TOKEN, EOS_TOKEN, UNK_TOKEN]

# Tokenizamos todos los titulares
tokenized_headings = [simple_tokenize(h) for h in df["Heading"]]

# Contamos frecuencia de palabras
counter = Counter()
for tokens in tokenized_headings:
    counter.update(tokens)

# Definimos un mínimo de frecuencia para incluir una palabra en el vocabulario
MIN_FREQ = 2

vocab = []
for word, freq in counter.items():
    if freq >= MIN_FREQ:
        vocab.append(word)

# Construimos mapping palabra->índice
itos = special_tokens + sorted(vocab)
stoi = {w:i for i, w in enumerate(itos)}

pad_idx = stoi[PAD_TOKEN]
bos_idx = stoi[BOS_TOKEN]
eos_idx = stoi[EOS_TOKEN]
unk_idx = stoi[UNK_TOKEN]

vocab_size = len(itos)
print("Tamaño del vocabulario:", vocab_size)
print("Ejemplo de palabras:", itos[:30])

Tamaño del vocabulario: 2304
Ejemplo de palabras: ['<pad>', '<bos>', '<eos>', '<unk>', '.', '0', '01', '1', '10', '100', '1000', '100m', '103', '104', '107', '108', '11', '117', '11m', '12', '120', '122', '126', '13', '130', '133', '14', '143', '145', '146']


In [ ]:
# Conversión de titulares a secuencias de índices
MAX_LEN = 20  # longitud máxima de secuencia (incluyendo <bos> y <eos>)

def encode_sentence(tokens, max_len=MAX_LEN):
    """
    Convierte una lista de tokens en una secuencia de índices con:
    <bos> tokens <eos> y padding hasta max_len.
    """
    ids = [bos_idx] + [stoi.get(t, unk_idx) for t in tokens] + [eos_idx]
    # Truncar si es demasiado larga
    ids = ids[:max_len]
    # Padding si es corta
    if len(ids) < max_len:
        ids += [pad_idx] * (max_len - len(ids))
    return ids

encoded_sequences = [encode_sentence(tokens) for tokens in tokenized_headings]
encoded_sequences = np.array(encoded_sequences)

print("Shape de encoded_sequences:", encoded_sequences.shape)
print("Ejemplo de secuencia codificada:", encoded_sequences[0])
print("Ejemplo de secuencia decodificada:",
      [itos[idx] for idx in encoded_sequences[0]])

Shape de encoded_sequences: (2692, 20)
Ejemplo de secuencia codificada: [   1 1856  895  576 2078  549 1582 2113  757  390    3    3    3    2
    0    0    0    0    0    0]
Ejemplo de secuencia decodificada: ['<bos>', 'sindh', 'govt', 'decides', 'to', 'cut', 'public', 'transport', 'fares', 'by', '<unk>', '<unk>', '<unk>', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [ ]:
# División en train / valid / test
N = len(encoded_sequences)
indices = np.arange(N)
np.random.shuffle(indices)

train_frac, val_frac = 0.7, 0.15
train_end = int(train_frac * N)
val_end = int((train_frac + val_frac) * N)

train_idx = indices[:train_end]
val_idx = indices[train_end:val_end]
test_idx = indices[val_end:]

train_data = encoded_sequences[train_idx]
val_data   = encoded_sequences[val_idx]
test_data  = encoded_sequences[test_idx]

print("Tamaño train:", train_data.shape)
print("Tamaño val:", val_data.shape)
print("Tamaño test:", test_data.shape)

Tamaño train: (1884, 20)
Tamaño val: (404, 20)
Tamaño test: (404, 20)


In [ ]:
# Dataset personalizado para lenguaje
class HeadlineLanguageDataset(Dataset):
    """
    Cada muestra es:
    - input_seq: [w0, w1, ..., w_{n-2}]
    - target_seq: [w1, w2, ..., w_{n-1}]
    (shift de 1 hacia la derecha)
    """
    def __init__(self, data_array):
        self.data = torch.tensor(data_array, dtype=torch.long)

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        seq = self.data[idx]
        # Input: todo menos el último token
        x = seq[:-1]
        # Target: todo menos el primero
        y = seq[1:]
        return x, y

BATCH_SIZE = 64

train_dataset = HeadlineLanguageDataset(train_data)
val_dataset   = HeadlineLanguageDataset(val_data)
test_dataset  = HeadlineLanguageDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

len(train_dataset), len(val_dataset), len(test_dataset)

(1884, 404, 404)

In [ ]:
# Definición del modelo LSTM de lenguaje
class LSTMLanguageModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers=1, dropout=0.0):
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.fc = nn.Linear(hidden_dim, vocab_size)

        self.init_weights()

    def init_weights(self):
        """
        Inicialización Xavier uniforme para embedding y capa lineal.
        Ayuda a una mejor propagación de gradientes al inicio del entrenamiento.
        """
        nn.init.xavier_uniform_(self.embedding.weight)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)

    def forward(self, x, hidden=None):
        # x: [batch, seq_len]
        emb = self.embedding(x)  # [batch, seq_len, embed_dim]
        out, hidden = self.lstm(emb, hidden)  # out: [batch, seq_len, hidden_dim]
        logits = self.fc(out)  # [batch, seq_len, vocab_size]
        return logits, hidden

In [ ]:
# Funciones de entrenamiento y evaluación
def calculate_perplexity(loss_value):
    """
    Calcula la perplejidad como exp(loss).
    """
    try:
        return math.exp(loss_value)
    except OverflowError:
        return float("inf")

def train_one_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    total_tokens = 0

    for x, y in dataloader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        logits, _ = model(x)
        # logits: [batch, seq_len, vocab_size]
        # y:      [batch, seq_len]
        # Reorganizamos para usar CrossEntropy
        logits = logits.reshape(-1, vocab_size)
        y = y.reshape(-1)

        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            # Contamos tokens no padding para el promedio
            mask = (y != pad_idx).float()
            total_loss += loss.item() * mask.sum().item()
            total_tokens += mask.sum().item()

    avg_loss = total_loss / total_tokens
    ppl = calculate_perplexity(avg_loss)
    return avg_loss, ppl

def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0.0
    total_tokens = 0

    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)

            logits, _ = model(x)
            logits = logits.reshape(-1, vocab_size)
            y = y.reshape(-1)

            loss = criterion(logits, y)

            mask = (y != pad_idx).float()
            total_loss += loss.item() * mask.sum().item()
            total_tokens += mask.sum().item()

    avg_loss = total_loss / total_tokens
    ppl = calculate_perplexity(avg_loss)
    return avg_loss, ppl

In [ ]:
# Entrenamiento de dos modelos (A y B)
EPOCHS = 8

def train_model_config(name, embed_dim, hidden_dim, num_layers, dropout=0.0, lr=1e-3):
    print(f"\n===== Entrenando {name} =====")
    model = LSTMLanguageModel(
        vocab_size=vocab_size,
        embed_dim=embed_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        dropout=dropout
    ).to(device)

    criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    history = {
        "train_loss": [],
        "train_ppl": [],
        "val_loss": [],
        "val_ppl": []
    }

    for epoch in range(1, EPOCHS + 1):
        train_loss, train_ppl = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_ppl = evaluate(model, val_loader, criterion)

        history["train_loss"].append(train_loss)
        history["train_ppl"].append(train_ppl)
        history["val_loss"].append(val_loss)
        history["val_ppl"].append(val_ppl)

        print(f"Epoch {epoch:02d} | "
              f"Train Loss: {train_loss:.4f}  PPL: {train_ppl:.2f} | "
              f"Val Loss: {val_loss:.4f}  PPL: {val_ppl:.2f}")

    return model, history

# Modelo A: pequeño
model_A, history_A = train_model_config(
    name="Modelo A (emb64, h128, 1 capa)",
    embed_dim=64,
    hidden_dim=128,
    num_layers=1,
    dropout=0.0,
    lr=1e-3
)

# Modelo B: más grande
model_B, history_B = train_model_config(
    name="Modelo B (emb128, h256, 2 capas, dropout)",
    embed_dim=128,
    hidden_dim=256,
    num_layers=2,
    dropout=0.3,
    lr=1e-3
)


===== Entrenando Modelo A (emb64, h128, 1 capa) =====
Epoch 01 | Train Loss: 7.3541  PPL: 1562.61 | Val Loss: 6.3990  PPL: 601.23
Epoch 02 | Train Loss: 6.2182  PPL: 501.82 | Val Loss: 6.2607  PPL: 523.56
Epoch 03 | Train Loss: 6.0558  PPL: 426.57 | Val Loss: 6.1911  PPL: 488.36
Epoch 04 | Train Loss: 5.9372  PPL: 378.86 | Val Loss: 6.0889  PPL: 440.94
Epoch 05 | Train Loss: 5.8007  PPL: 330.53 | Val Loss: 5.9778  PPL: 394.59
Epoch 06 | Train Loss: 5.6892  PPL: 295.65 | Val Loss: 5.9221  PPL: 373.18
Epoch 07 | Train Loss: 5.6098  PPL: 273.10 | Val Loss: 5.8727  PPL: 355.19
Epoch 08 | Train Loss: 5.5429  PPL: 255.42 | Val Loss: 5.8492  PPL: 346.95

===== Entrenando Modelo B (emb128, h256, 2 capas, dropout) =====
Epoch 01 | Train Loss: 6.8237  PPL: 919.39 | Val Loss: 6.3022  PPL: 545.75
Epoch 02 | Train Loss: 6.0902  PPL: 441.52 | Val Loss: 6.1696  PPL: 477.99
Epoch 03 | Train Loss: 5.8866  PPL: 360.19 | Val Loss: 6.0111  PPL: 407.94
Epoch 04 | Train Loss: 5.7400  PPL: 311.07 | Val Loss

In [ ]:
# Función para generar titulares con el modelo
def generate_headline(model, max_len=15, temperature=1.0, start_token=BOS_TOKEN):
    """
    Genera un titular palabra por palabra usando greedy sampling con temperatura.
    """
    model.eval()
    with torch.no_grad():
        # Comenzamos con <bos>
        input_ids = torch.tensor([[stoi[start_token]]], dtype=torch.long).to(device)
        generated = [start_token]

        hidden = None
        for _ in range(max_len):
            logits, hidden = model(input_ids, hidden)
            # Tomamos el último paso de la secuencia
            logits_step = logits[:, -1, :] / temperature
            probs = torch.softmax(logits_step, dim=-1)
            # Elegimos la palabra con mayor probabilidad
            next_id = torch.multinomial(probs, num_samples=1).item()

            next_token = itos[next_id]
            if next_token == EOS_TOKEN:
                break
            generated.append(next_token)

            # El siguiente input es el token recién generado
            input_ids = torch.tensor([[next_id]], dtype=torch.long).to(device)

        # Unimos en un string ignorando <bos>
        return " ".join(generated[1:])

# Ejemplo de uso con el modelo B
for i in range(5):
    print(f"TITULAR GENERADO {i+1}: {generate_headline(model_B, max_len=12, temperature=0.9)}")

TITULAR GENERADO 1: summer leads us reach against
TITULAR GENERADO 2: pcb <unk> <unk> in bounce to 1 bln day
TITULAR GENERADO 3: gladiators leaves economy india in on <unk>
TITULAR GENERADO 4: messi shares gas delays to zone last second <unk>
TITULAR GENERADO 5: sbp heat till <unk> online at doping day first


In [ ]:
# Evaluación final en conjunto de prueba
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

test_loss_A, test_ppl_A = evaluate(model_A, test_loader, criterion)
test_loss_B, test_ppl_B = evaluate(model_B, test_loader, criterion)

print("===== Resultados en TEST =====")
print(f"Modelo A - Test Loss: {test_loss_A:.4f} | Test PPL: {test_ppl_A:.2f}")
print(f"Modelo B - Test Loss: {test_loss_B:.4f} | Test PPL: {test_ppl_B:.2f}")


===== Resultados en TEST =====
Modelo A - Test Loss: 5.8343 | Test PPL: 341.83
Modelo B - Test Loss: 5.7743 | Test PPL: 321.92
